In [ ]:
using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles, PottsGauge
using AncestralSequenceReconstruction # core package
using JLD2 # used to load ArDCA models
using TreeTools # to handle p
using ArDCA, KitMSA
using Distributions



nat_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);
w = compute_weights(nat_msa, 22, 0.2)[1];
f1_nat, f2_nat = compute_weighted_frequencies(Int8.(nat_msa),22, 0.); c= reshape(
    f2_nat - f1_nat*f1_nat', q, L, q, L);

@load "../data_Genie/pars_dbd.jld2"; h = h_dbd; J = J_dbd;


file_seqs = "../res_algos_old/alpha0.5_potts_mu10.0.fa"

msa = read_fasta_alignment(file_seqs, 0.9); L, M = size(msa); q = 21;


ens = []; cors = []; reshuf_msa = Int.(zeros(L,M,10)); ts = [0.,0.2,0.5, 1., 1.2, 1.5, 5.]; reshuf_msa[:,:,1] .= msa

f1, f2 = compute_weighted_frequencies(Int8.(reshuf_msa[:,:,1]),22, 0.); c= reshape(
    f2 - f1*f1', q, L, q, L);

push!(ens, PhyloTools.energy(reshuf_msa[:,:,1], h, J)); push!(cors,c); 



for i in 2:length(ts)
    
    reshuf_msa[:,:,i] .= PhyloTools.reshuffle_entr(reshuf_msa[:,:,i-1], h, J, temp = 0.6, times = ts[i]-ts[i-1]);
    
    f1, f2 = compute_weighted_frequencies(Int8.(reshuf_msa[:,:,i]),22, 0.); c= reshape(
    f2 - f1*f1', q, L, q, L);

    push!(ens, PhyloTools.energy(reshuf_msa[:,:,i], h, J)); push!(cors,c); 
    
end

m_c = [mean(cors[i]) for i in 1:length(ts)]
s_c = [std(cors[i]) for i in 1:length(ts)]
max_c = 
    
c = cors[end]; c_0 = cors[1]; 
indices = partialsortperm(J_dbd[:], 1:1000, rev=true);

close("all"); plt.scatter(c_0[indices], c[indices]);plt.plot(
    [minimum(c_0[indices]), maximum(c_0[indices])],[minimum(c_0[indices]), maximum(c_0[indices])]
    ); savefig("../trial_real.png")

close("all"); plt.hist(c[:],density = true, histtype="step",label = "reshuf"); plt.hist(c_0[:],
    density = true,histtype = "step",label = "random"); plt.legend();savefig("../hist_real.png")

[mean(ens[i]) for i in 1:length(ens)]

